<a href="https://colab.research.google.com/github/rwlankford/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data
x_columns =  ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[x_columns]

# Show the data types for X_df
X_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [5]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'OverTime' column in X_df using .loc
X_df.loc[:, 'OverTime'] = label_encoder.fit_transform(X_df['OverTime'])

# Display the counts of each label
X_df['OverTime'].value_counts()


OverTime
0    1054
1     416
Name: count, dtype: int64

In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [8]:
# Create a OneHotEncoder for the Department column
department_ohe = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_ohe.fit(y_train['Department'].values.reshape(-1, 1))

# Create two new variables by applying the encoder to the training and testing data
y_train_encoded_department = department_ohe.transform(y_train['Department'].values.reshape(-1, 1))
y_test_encoded_department = department_ohe.transform(y_test['Department'].values.reshape(-1, 1))


y_test_encoded_department[:5]


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [10]:
# Create a OneHotEncoder for the Attrition column
attrition_ohe = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_ohe.fit(y_train['Attrition'].values.reshape(-1, 1))

# Create two new variables by applying the encoder to the training and testing data
y_train_encoded_attrition = attrition_ohe.transform(y_train['Attrition'].values.reshape(-1, 1))
y_test_encoded_attrition = attrition_ohe.transform(y_test['Attrition'].values.reshape(-1, 1))

y_test_encoded_attrition[:5]



array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [11]:
# Find the number of columns in the X training data
number_input_features = X_train_scaled.shape[1]

# Create the input layer
input = layers.Input(shape=(number_input_features,), name='input')

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu', name='shared1')(input)
shared2 = layers.Dense(128, activation='relu', name='shared2')(shared1)


In [12]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)



In [13]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared2)

# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', name='attrition_output')(attrition_hidden)



In [14]:
# Create the model
model = Model(inputs=input, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 10)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   704       ['input[0][0]']               
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                              

In [20]:
# Train the model
model.fit(X_train_scaled, {'department_output': y_train_encoded_department, 'attrition_output': y_train_encoded_attrition}, epochs=100, shuffle=True, verbose=2, batch_size=32)


Epoch 1/100
35/35 - 0s - loss: 0.0495 - department_output_loss: 0.0328 - attrition_output_loss: 0.0167 - department_output_accuracy: 0.9936 - attrition_output_accuracy: 0.9964 - 172ms/epoch - 5ms/step
Epoch 2/100
35/35 - 0s - loss: 0.0459 - department_output_loss: 0.0314 - attrition_output_loss: 0.0145 - department_output_accuracy: 0.9973 - attrition_output_accuracy: 0.9973 - 156ms/epoch - 4ms/step
Epoch 3/100
35/35 - 0s - loss: 0.0458 - department_output_loss: 0.0325 - attrition_output_loss: 0.0133 - department_output_accuracy: 0.9964 - attrition_output_accuracy: 0.9982 - 179ms/epoch - 5ms/step
Epoch 4/100
35/35 - 0s - loss: 0.0392 - department_output_loss: 0.0262 - attrition_output_loss: 0.0130 - department_output_accuracy: 0.9964 - attrition_output_accuracy: 0.9955 - 157ms/epoch - 4ms/step
Epoch 5/100
35/35 - 0s - loss: 0.0339 - department_output_loss: 0.0254 - attrition_output_loss: 0.0085 - department_output_accuracy: 0.9982 - attrition_output_accuracy: 0.9991 - 243ms/epoch - 7ms/

In [21]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department_output': y_test_encoded_department, 'attrition_output': y_test_encoded_attrition},  verbose=2)
test_results

12/12 - 0s - loss: 6.7802 - department_output_loss: 4.6259 - attrition_output_loss: 2.1543 - department_output_accuracy: 0.5408 - attrition_output_accuracy: 0.7772 - 44ms/epoch - 4ms/step


[6.780197620391846,
 4.625884532928467,
 2.154313325881958,
 0.54076087474823,
 0.7771739363670349]

In [22]:
# Print the accuracy for both department and attrition
print(f"Department Predictions Accuracy: {test_results[3]}")
print(f"Attrition Predictions Accuracy: {test_results[4]}")

Department Predictions Accuracy: 0.54076087474823
Attrition Predictions Accuracy: 0.7771739363670349


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No.  The reason why it is not the best metric to use on this data is that the data could be imbalanced.  An imbalance would cause the model to have high accuracy but low performance on the task.
2. For 'Attrition' I used Sigmoid because the data had a binary classification.  For 'Department' I used Softmax because it was a multi-class classification.
3. One way the model could be imporoved is by encoding categorical variables more effectively. Another way the model could be improved is by making it more compelex whether it is through adding more laters and neurons or using different models like Random Forest. Other ways include class weights, data augmentation, and other techniques.